<a href="https://colab.research.google.com/github/JoelPasapera/AI-Strategies-in-python-google-colab-/blob/main/MODELO_IA_para_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
from pathlib import Path

# Crea una carpeta (si no existe) en el google drive llamada 'Colab'
ruta_drive: Path = Path('/content/drive/MyDrive/Colab/AI_MODEL')
# En esta carpeta se guardarán todos los progresos para no perder nada
ruta_drive.mkdir(parents=True, exist_ok=True)

# Se define la ruta de los checkpoints
checkpoint_path = Path(ruta_drive, "checkpoints")
checkpoint_path.mkdir(parents=True, exist_ok=True)

# Se define la ruta del archivo CSV
csv_path: Path = Path(ruta_drive, "GBPUSD_16385_minutes.csv")

# Se define la ruta para el modelo guardado
model_path: Path = Path(ruta_drive, "best_trading_model.h5")

# Se define la ruta para los registros (logs)
logs_path: Path = Path(ruta_drive,'logs')

def comprobar_rutas(*rutas):
    """Retorna diccionario con nombres y bool si existe ruta"""
    resultados = {}
    for r in rutas:
        ruta = Path(r)
        existe = ruta.exists()
        resultados[str(ruta)] = existe

        if existe:
            print(f"✔ La ruta existe: {ruta}")
        else:
            print(f"✘ La ruta NO existe: {ruta}")

    return resultados


comprobar_rutas(csv_path, model_path)

# Ejemplo creando archivo en la carpeta de destino
ruta_archivo: Path = Path(ruta_drive, "mi_archivo.txt")
with open(ruta_archivo, "w") as f:
    f.write("Hola, este archivo se guardó directamente en Drive!")

✔ La ruta existe: /content/drive/MyDrive/Colab/AI_MODEL/GBPUSD_16385_minutes.csv
✔ La ruta existe: /content/drive/MyDrive/Colab/AI_MODEL/best_trading_model.h5


In [ ]:
# =============================================
# CONFIGURACIÓN INICIAL Y INSTALACIÓN DE LIBRERÍAS
# =============================================
%%capture
!pip install tensorflow #==2.13.0
!pip install keras-tuner
!pip install ta
!pip install plotly
!pip install yfinance
!pip install scikit-optimize

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =============================================
# CARGA Y PREPROCESAMIENTO DE DATOS
# =============================================

# Cargar datos
df = pd.read_csv(csv_path, encoding='utf-16')

print(f"Dataset cargado: {df.shape}")
print(df.head())

# Procesamiento de fecha
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d %H:%M:%S')
df = df.sort_values('date').reset_index(drop=True)

Dataset cargado: (67662, 5)
                  date     open     high      low    close
0  2025.11.14 23:00:00  1.31677  1.31745  1.31645  1.31744
1  2025.11.14 22:00:00  1.31646  1.31707  1.31614  1.31677
2  2025.11.14 21:00:00  1.31603  1.31676  1.31561  1.31646
3  2025.11.14 20:00:00  1.31608  1.31632  1.31493  1.31603
4  2025.11.14 19:00:00  1.31548  1.31630  1.31507  1.31609


In [ ]:
# ============================================
# CONFIGURACIONES GENERALES
# ============================================
# -*- coding: utf-8 -*-
"""Modelo Trading para GBPUSD.ipynb

Automatically generated by Colaboratory.

Original file is located at:
    https://colab.research.google.com/drive/1abc123xyz
"""

# ============================================
# PARÁMETROS CONFIGURABLES
# ============================================
EPOCHS: int = 200
BATCH_SIZE: int = 2560
SEQUENCE_LENGTH: int = 60
CHECKPOINT_INTERVAL: int = 10  # Guardar cada 10 épocas
INITIAL_EPOCH: int = 0  # Empezar desde 0 o continuar desde checkpoint
RESUME_TRAINING: bool = True  # Continuar entrenamiento desde checkpoint

# Hiperparámetros del modelo
LEARNING_RATE: float = 0.001
DROPOUT_RATE: float = 0.3
L2_REGULARIZATION: float = 1e-4

# Configuración para máxima performance
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True)

# Verificar GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPUs disponibles")
    except RuntimeError as e:
        print(e)

# =============================================
# ANÁLISIS EXPLORATORIO Y FEATURE ENGINEERING
# =============================================

import ta
from ta import add_all_ta_features
from ta.utils import dropna

class AdvancedFeatureEngineer:
    def __init__(self, df):
        self.df = df.copy()

    def add_price_features(self):
        """Características basadas en precio"""
        df = self.df

        # Returns y volatilidad
        df['returns'] = df['close'].pct_change()
        df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
        df['volatility'] = df['returns'].rolling(window=20).std()
        df['price_range'] = (df['high'] - df['low']) / df['close']
        df['body_size'] = abs(df['close'] - df['open']) / df['close']

        # Medias móviles
        for window in [5, 10, 20, 50]:
            df[f'sma_{window}'] = df['close'].rolling(window=window).mean()
            df[f'ema_{window}'] = df['close'].ewm(span=window).mean()
            df[f'price_vs_sma_{window}'] = df['close'] / df[f'sma_{window}'] - 1

        return df

    def add_technical_indicators(self):
        """Indicadores técnicos avanzados"""
        df = self.df

        # RSI
        df['rsi'] = ta.momentum.RSIIndicator(df['close']).rsi()

        # MACD
        macd = ta.trend.MACD(df['close'])
        df['macd'] = macd.macd()
        df['macd_signal'] = macd.macd_signal()
        df['macd_histogram'] = macd.macd_diff()

        # Bollinger Bands
        bollinger = ta.volatility.BollingerBands(df['close'])
        df['bb_upper'] = bollinger.bollinger_hband()
        df['bb_lower'] = bollinger.bollinger_lband()
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['close']
        df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])

        # Ichimoku Cloud
        ichimoku = ta.trend.IchimokuIndicator(df['high'], df['low'])
        df['ichimoku_a'] = ichimoku.ichimoku_a()
        df['ichimoku_b'] = ichimoku.ichimoku_b()
        df['ichimoku_base'] = ichimoku.ichimoku_base_line()
        df['ichimoku_conversion'] = ichimoku.ichimoku_conversion_line()

        # Volumen (aunque no tenemos volumen, usamos rangos de precio como proxy)
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()

        # Stochastic
        stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
        df['stoch_k'] = stoch.stoch()
        df['stoch_d'] = stoch.stoch_signal()

        # ADX
        df['adx'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close']).adx()

        return df

    def add_time_features(self):
        """Características temporales"""
        df = self.df

        df['hour'] = df['date'].dt.hour
        df['day_of_week'] = df['date'].dt.dayofweek
        df['day_of_month'] = df['date'].dt.day
        df['month'] = df['date'].dt.month
        df['week_of_year'] = df['date'].dt.isocalendar().week

        # Ciclos temporales
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

        return df

    def add_statistical_features(self):
        """Características estadísticas avanzadas"""
        df = self.df

        # Rolling statistics
        for window in [10, 20, 50]:
            df[f'rolling_mean_{window}'] = df['close'].rolling(window).mean()
            df[f'rolling_std_{window}'] = df['close'].rolling(window).std()
            df[f'rolling_skew_{window}'] = df['close'].rolling(window).skew()
            df[f'rolling_kurt_{window}'] = df['close'].rolling(window).kurt()

        # Z-score
            df[f'z_score_{window}'] = (df['close'] - df[f'rolling_mean_{window}']) / df[f'rolling_std_{window}']

        # Hurst Exponent (simplificado)
        def hurst_exponent(ts):
            lags = range(2, 20)
            tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
            poly = np.polyfit(np.log(lags), np.log(tau), 1)
            return poly[0]

        df['hurst_20'] = df['close'].rolling(100).apply(hurst_exponent, raw=True)

        return df

    def add_target_variables(self):
        """Variables objetivo para el modelo"""
        df = self.df

        # Retornos futuros (1h, 4h, 24h)
        df['target_1h'] = df['close'].shift(-1) / df['close'] - 1
        df['target_4h'] = df['close'].shift(-4) / df['close'] - 1
        df['target_24h'] = df['close'].shift(-24) / df['close'] - 1

        # Dirección del mercado
        df['target_direction_1h'] = np.where(df['target_1h'] > 0, 1, 0)
        df['target_direction_4h'] = np.where(df['target_4h'] > 0, 1, 0)

        # Volatilidad futura
        df['target_volatility_4h'] = df['returns'].shift(-4).rolling(4).std()

        return df

    def engineer_all_features(self):
        """Ejecutar todo el feature engineering"""
        df = self.add_price_features()
        df = self.add_technical_indicators()
        df = self.add_time_features()
        df = self.add_statistical_features()
        df = self.add_target_variables()

        # Eliminar filas con NaN
        df = df.dropna()

        return df

# Aplicar feature engineering
feature_engineer = AdvancedFeatureEngineer(df)
df_enhanced = feature_engineer.engineer_all_features()

print(f"Dataset después del feature engineering: {df_enhanced.shape}")

# =============================================
# PREPARACIÓN DE DATOS PARA EL MODELO
# =============================================

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

class DataPreprocessor:
    def __init__(self, df, sequence_length=SEQUENCE_LENGTH, test_size=0.2, val_size=0.1):
        self.df = df
        self.sequence_length = sequence_length
        self.test_size = test_size
        self.val_size = val_size
        self.scalers = {}

    def prepare_features(self):
        """Seleccionar y preparar características"""
        # Excluir columnas no numéricas y targets
        exclude_cols = ['date', 'target_1h', 'target_4h', 'target_24h',
                       'target_direction_1h', 'target_direction_4h', 'target_volatility_4h']

        feature_cols = [col for col in self.df.columns if col not in exclude_cols]

        # Separar características y targets
        X = self.df[feature_cols].values
        y_regression = self.df[['target_1h', 'target_4h', 'target_24h']].values
        y_classification = self.df[['target_direction_1h', 'target_direction_4h']].values

        return X, y_regression, y_classification, feature_cols

    def create_sequences(self, X, y_reg, y_clf):
        """Crear secuencias para LSTM"""
        X_seq, y_reg_seq, y_clf_seq = [], [], []

        for i in range(self.sequence_length, len(X)):
            X_seq.append(X[i-self.sequence_length:i])
            y_reg_seq.append(y_reg[i])
            y_clf_seq.append(y_clf[i])

        return np.array(X_seq), np.array(y_reg_seq), np.array(y_clf_seq)

    def scale_features(self, X_train, X_val, X_test):
        """Escalar características de forma robusta"""
        # RobustScaler es mejor para datos financieros con outliers
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
        X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1]))
        X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1]))

        # Reshape back to sequences
        X_train_scaled = X_train_scaled.reshape(X_train.shape)
        X_val_scaled = X_val_scaled.reshape(X_val.shape)
        X_test_scaled = X_test_scaled.reshape(X_test.shape)

        self.scalers['feature'] = scaler
        return X_train_scaled, X_val_scaled, X_test_scaled

    def prepare_datasets(self):
        """Preparar todos los datasets"""
        X, y_reg, y_clf, feature_cols = self.prepare_features()

        # Crear secuencias
        X_seq, y_reg_seq, y_clf_seq = self.create_sequences(X, y_reg, y_clf)

        # Split temporal (importante para time series)
        split_idx1 = int(len(X_seq) * (1 - self.test_size - self.val_size))
        split_idx2 = int(len(X_seq) * (1 - self.test_size))

        X_train, X_val, X_test = (X_seq[:split_idx1],
                                 X_seq[split_idx1:split_idx2],
                                 X_seq[split_idx2:])

        y_reg_train, y_reg_val, y_reg_test = (y_reg_seq[:split_idx1],
                                            y_reg_seq[split_idx1:split_idx2],
                                            y_reg_seq[split_idx2:])

        y_clf_train, y_clf_val, y_clf_test = (y_clf_seq[:split_idx1],
                                            y_clf_seq[split_idx1:split_idx2],
                                            y_clf_seq[split_idx2:])

        # Escalar características
        X_train_scaled, X_val_scaled, X_test_scaled = self.scale_features(X_train, X_val, X_test)

        print(f"Training set: {X_train_scaled.shape}")
        print(f"Validation set: {X_val_scaled.shape}")
        print(f"Test set: {X_test_scaled.shape}")

        return (X_train_scaled, X_val_scaled, X_test_scaled,
                y_reg_train, y_reg_val, y_reg_test,
                y_clf_train, y_clf_val, y_clf_test,
                feature_cols)

# Preparar datos
preprocessor = DataPreprocessor(df_enhanced, sequence_length=SEQUENCE_LENGTH)
(X_train, X_val, X_test,
 y_reg_train, y_reg_val, y_reg_test,
 y_clf_train, y_clf_val, y_clf_test,
 feature_cols) = preprocessor.prepare_datasets()

# =============================================
# ARQUITECTURA DEL MODELO NEURONAL AVANZADO
# =============================================

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (LSTM, Dense, Input, Conv1D, MaxPooling1D,
                                   Flatten, Dropout, BatchNormalization,
                                   Attention, MultiHeadAttention, GlobalAveragePooling1D,
                                   concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ReduceLROnPlateau, EarlyStopping,
                                      ModelCheckpoint, TensorBoard, Callback)
from tensorflow.keras.regularizers import l2
import pickle
import os

# =============================================
# SISTEMA DE CHECKPOINT MEJORADO
# =============================================

class AdvancedCheckpoint(Callback):
    """Callback avanzado para guardar checkpoints periódicos"""

    def __init__(self, checkpoint_path=checkpoint_path, interval=CHECKPOINT_INTERVAL,
                 save_optimizer=True, max_to_keep=5):
        super().__init__()
        self.checkpoint_path = checkpoint_path
        self.interval = interval
        self.save_optimizer = save_optimizer
        self.max_to_keep = max_to_keep
        os.makedirs(checkpoint_path, exist_ok=True)
        self.training_history = {}

    def on_epoch_end(self, epoch, logs=None):
        """Registro + Guardado del checkpoint"""
        # ========== Registrar historial ==========
        if logs:
            for k, v in logs.items():
                if k not in self.training_history:
                    self.training_history[k] = []
                self.training_history[k].append(v)

        # ========== Guardar checkpoint periódico ==========
        if (epoch + 1) % self.interval == 0:
            # Guardar modelo
            model_path = os.path.join(self.checkpoint_path, f'model_epoch_{epoch+1}.h5')
            self.model.save(model_path)

            # Guardar historial de entrenamiento
            history_path = os.path.join(self.checkpoint_path, f'training_history.pkl')
            if hasattr(self, 'training_history'):
                with open(history_path, 'wb') as f:
                    pickle.dump(self.training_history, f)

            # Guardar info del checkpoint
            checkpoint_info = {
                'epoch': epoch + 1,
                'logs': logs,
                'timestamp': datetime.now().isoformat(),
                'model_path': model_path,
                'total_epochs': EPOCHS,
                'batch_size': BATCH_SIZE,
                'learning_rate': LEARNING_RATE
            }

            info_path = os.path.join(self.checkpoint_path, f'checkpoint_info_epoch_{epoch+1}.pkl')
            with open(info_path, 'wb') as f:
                pickle.dump(checkpoint_info, f)

            # Limpiar checkpoints antiguos
            self._clean_old_checkpoints(epoch + 1)

            print(f"✅ Checkpoint guardado en época {epoch + 1}")

    def on_epoch_end(self, epoch, logs=None):
        # Actualizar historial
        for k, v in logs.items():
            if k not in self.training_history:
                self.training_history[k] = []
            self.training_history[k].append(v)

        # Guardar checkpoint periódico
        if (epoch + 1) % self.interval == 0:
            self._save_checkpoint(epoch, logs)

    def _save_checkpoint(self, epoch, logs):
        """Guardar checkpoint completo"""
        # Guardar modelo
        model_path = os.path.join(self.checkpoint_path, f'model_epoch_{epoch+1}.h5')
        self.model.save(model_path)

        # Guardar historial
        history_path = os.path.join(self.checkpoint_path, 'training_history.pkl')
        with open(history_path, 'wb') as f:
            pickle.dump(self.training_history, f)

        # Guardar información del checkpoint
        checkpoint_info = {
            'epoch': epoch + 1,
            'logs': logs,
            'timestamp': datetime.now().isoformat(),
            'model_path': model_path,
            'total_epochs': EPOCHS,
            'batch_size': BATCH_SIZE,
            'learning_rate': LEARNING_RATE
        }

        info_path = os.path.join(self.checkpoint_path, f'checkpoint_info_epoch_{epoch+1}.pkl')
        with open(info_path, 'wb') as f:
            pickle.dump(checkpoint_info, f)

        print(f"✅ Checkpoint guardado en época {epoch + 1}")

    def _clean_old_checkpoints(self, current_epoch):
        """Eliminar checkpoints antiguos"""
        checkpoints = []
        for f in os.listdir(self.checkpoint_path):
            if f.startswith('model_epoch_') and f.endswith('.h5'):
                epoch_num = int(f.split('_')[2].split('.')[0])
                checkpoints.append((epoch_num, f))

        # Ordenar y mantener solo los más recientes
        checkpoints.sort(reverse=True)
        for epoch_num, filename in checkpoints[self.max_to_keep:]:
            file_path = os.path.join(self.checkpoint_path, filename)
            info_path = os.path.join(self.checkpoint_path, f'checkpoint_info_epoch_{epoch_num}.pkl')

            if os.path.exists(file_path):
                os.remove(file_path)
            if os.path.exists(info_path):
                os.remove(info_path)

def find_latest_checkpoint(checkpoint_path=checkpoint_path):
    """Encontrar el checkpoint más reciente"""
    if not os.path.exists(checkpoint_path):
        return None, 0, None

    checkpoints = []
    for f in os.listdir(checkpoint_path):
        if f.startswith('model_epoch_') and f.endswith('.h5'):
            epoch_num = int(f.split('_')[2].split('.')[0])
            checkpoints.append((epoch_num, f))

    if not checkpoints:
        return None, 0, None

    # Encontrar el más reciente
    latest_epoch, latest_file = max(checkpoints, key=lambda x: x[0])
    latest_path = os.path.join(checkpoint_path, latest_file)

    # Cargar historial si existe
    history_path = os.path.join(checkpoint_path, 'training_history.pkl')
    history = None
    if os.path.exists(history_path):
        with open(history_path, 'rb') as f:
            history = pickle.load(f)

    return latest_path, latest_epoch, history

def load_model_from_checkpoint(model_path, strategy):
    """
    Intenta cargar el .h5 normalmente. Si falla por deserialización de losses/metrics
    hace fallback a load_model(..., compile=False) y recompila con los valores esperados.
    """
    from tensorflow.keras.optimizers import Adam

    with strategy.scope():
        try:
            # Intento normal (puede fallar si hay incompatibilidades)
            model = tf.keras.models.load_model(model_path)
            print("Modelo cargado (compile=True).")
            return model
        except Exception as e:
            print("⚠️ Falló la carga normal del modelo:", e)
            print("➡️ Intentando fallback: cargar con compile=False y recompilar manualmente...")

            # Cargar sin compilar (evita la deserialización de losses/metrics)
            model = tf.keras.models.load_model(model_path, compile=False)

            # RECOMPILAR: usar exactamente las losses/weights/metrics/optimizer que usas al entrenar
            loss_dict = {
                'return_1h': 'mse',
                'return_4h': 'mse',
                'return_24h': 'mse',
                'direction_1h': 'binary_crossentropy',
                'direction_4h': 'binary_crossentropy'
            }

            loss_weights = {
                'return_1h': 1.0,
                'return_4h': 1.0,
                'return_24h': 0.5,
                'direction_1h': 2.0,
                'direction_4h': 1.5
            }

            metrics_dict = {
                'return_1h': ['mae', 'mse'],
                'return_4h': ['mae', 'mse'],
                'return_24h': ['mae', 'mse'],
                'direction_1h': ['accuracy', 'binary_accuracy'],
                'direction_4h': ['accuracy', 'binary_accuracy']
            }

            # Recreate optimizer (ajusta LR si corresponde)
            optimizer = Adam(learning_rate=LEARNING_RATE, clipvalue=1.0)

            model.compile(optimizer=optimizer,
                          loss=loss_dict,
                          loss_weights=loss_weights,
                          metrics=metrics_dict)

            print("✅ Modelo recompilado con compile=False -> compile(...)")
            return model

class AdvancedTradingModel:
    def __init__(self, sequence_length, n_features):
        self.sequence_length = sequence_length
        self.n_features = n_features

    def build_hybrid_model(self):
        """Modelo híbrido CNN-LSTM-Attention"""
        # Input
        input_layer = Input(shape=(self.sequence_length, self.n_features))

        # CNN Branch para patrones locales
        conv1 = Conv1D(filters=64, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(L2_REGULARIZATION))(input_layer)
        conv1 = BatchNormalization()(conv1)
        conv1 = MaxPooling1D(pool_size=2)(conv1)

        conv2 = Conv1D(filters=128, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(L2_REGULARIZATION))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = MaxPooling1D(pool_size=2)(conv2)

        conv3 = Conv1D(filters=256, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(L2_REGULARIZATION))(conv2)
        conv3 = BatchNormalization()(conv3)

        # LSTM Branch para dependencias temporales
        lstm1 = LSTM(256, return_sequences=True,
                    kernel_regularizer=l2(L2_REGULARIZATION))(input_layer)
        lstm1 = BatchNormalization()(lstm1)
        lstm1 = Dropout(DROPOUT_RATE)(lstm1)

        lstm2 = LSTM(128, return_sequences=True,
                    kernel_regularizer=l2(L2_REGULARIZATION))(lstm1)
        lstm2 = BatchNormalization()(lstm2)
        lstm2 = Dropout(DROPOUT_RATE)(lstm2)

        # Attention mechanism
        attention = MultiHeadAttention(num_heads=8, key_dim=64)(lstm2, lstm2)
        attention = BatchNormalization()(attention)

        # Combine CNN and LSTM branches
        cnn_flat = GlobalAveragePooling1D()(conv3)
        lstm_flat = GlobalAveragePooling1D()(attention)

        # Concatenate all branches
        concatenated = concatenate([cnn_flat, lstm_flat])

        # Dense layers
        dense1 = Dense(512, activation='relu', kernel_regularizer=l2(L2_REGULARIZATION))(concatenated)
        dense1 = BatchNormalization()(dense1)
        dense1 = Dropout(DROPOUT_RATE + 0.1)(dense1)

        dense2 = Dense(256, activation='relu', kernel_regularizer=l2(L2_REGULARIZATION))(dense1)
        dense2 = BatchNormalization()(dense2)
        dense2 = Dropout(DROPOUT_RATE + 0.1)(dense2)

        dense3 = Dense(128, activation='relu', kernel_regularizer=l2(L2_REGULARIZATION))(dense2)
        dense3 = BatchNormalization()(dense3)
        dense3 = Dropout(DROPOUT_RATE)(dense3)

        # Multi-output
        # Regression outputs
        reg_output1 = Dense(1, activation='linear', name='return_1h')(dense3)
        reg_output2 = Dense(1, activation='linear', name='return_4h')(dense3)
        reg_output3 = Dense(1, activation='linear', name='return_24h')(dense3)

        # Classification outputs
        clf_output1 = Dense(1, activation='sigmoid', name='direction_1h')(dense3)
        clf_output2 = Dense(1, activation='sigmoid', name='direction_4h')(dense3)

        model = Model(inputs=input_layer,
                     outputs=[reg_output1, reg_output2, reg_output3,
                             clf_output1, clf_output2])

        return model

# =============================================
# CONFIGURACIÓN DEL ENTRENAMIENTO
# =============================================

# Estrategia distribuida
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Verificar si hay checkpoint previo
initial_epoch = INITIAL_EPOCH
history_object = None

if RESUME_TRAINING:
    latest_checkpoint, latest_epoch, history_object = find_latest_checkpoint()
    if latest_checkpoint:
        print(f"🔄 Checkpoint encontrado: {latest_checkpoint}")
        print(f"📅 Continuando desde época: {latest_epoch}")
        initial_epoch = latest_epoch

        # Cargar modelo desde checkpoint
        with strategy.scope():
            model = load_model_from_checkpoint(latest_checkpoint, strategy)
        print("✅ Modelo cargado desde checkpoint")
    else:
        print("🚀 No se encontraron checkpoints. Iniciando entrenamiento desde cero.")
        # Construir nuevo modelo
        with strategy.scope():
            model_builder = AdvancedTradingModel(sequence_length=SEQUENCE_LENGTH, n_features=X_train.shape[2])
            model = model_builder.build_hybrid_model()
else:
    print("🚀 Iniciando entrenamiento nuevo (RESUME_TRAINING=False)")
    with strategy.scope():
        model_builder = AdvancedTradingModel(sequence_length=SEQUENCE_LENGTH, n_features=X_train.shape[2])
        model = model_builder.build_hybrid_model()

# Compilar modelo (solo si no está compilado o si estamos empezando desde 0)
if initial_epoch == 0:
    with strategy.scope():
        model.compile(
            optimizer=Adam(learning_rate=LEARNING_RATE, clipvalue=1.0),
            loss={
                'return_1h': 'mse',
                'return_4h': 'mse',
                'return_24h': 'mse',
                'direction_1h': 'binary_crossentropy',
                'direction_4h': 'binary_crossentropy'
            },
            loss_weights={
                'return_1h': 1.0,
                'return_4h': 1.0,
                'return_24h': 0.5,
                'direction_1h': 2.0,
                'direction_4h': 1.5
            },
            metrics={
                'return_1h': ['mae', 'mse'],
                'return_4h': ['mae', 'mse'],
                'return_24h': ['mae', 'mse'],
                'direction_1h': ['accuracy', 'binary_accuracy'],
                'direction_4h': ['accuracy', 'binary_accuracy']
            }
        )
    print("✅ Modelo compilado exitosamente")

# Callbacks
advanced_checkpoint = AdvancedCheckpoint(interval=CHECKPOINT_INTERVAL)

callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        model_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    TensorBoard(
        log_dir=logs_path,
        histogram_freq=1,
        update_freq='epoch'
    ),
    advanced_checkpoint
]

# Si hay historial previo, cargarlo en el callback
if history_object and hasattr(advanced_checkpoint, 'training_history'):
    advanced_checkpoint.training_history = history_object

print(f"🎯 Configuración de entrenamiento:")
print(f"   - Épocas totales: {EPOCHS}")
print(f"   - Época inicial: {initial_epoch}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Checkpoint cada: {CHECKPOINT_INTERVAL} épocas")
print(f"   - Learning rate: {LEARNING_RATE}")

# =============================================
# ENTRENAMIENTO DEL MODELO
# =============================================

# Solo entrenar si no hemos llegado al final
if initial_epoch < EPOCHS:
    print("🚀 Iniciando entrenamiento...")

    history = model.fit(
        X_train,
        {
            'return_1h': y_reg_train[:, 0],
            'return_4h': y_reg_train[:, 1],
            'return_24h': y_reg_train[:, 2],
            'direction_1h': y_clf_train[:, 0],
            'direction_4h': y_clf_train[:, 1]
        },
        batch_size=BATCH_SIZE,
        initial_epoch=initial_epoch,
        epochs=EPOCHS,
        validation_data=(
            X_val,
            {
                'return_1h': y_reg_val[:, 0],
                'return_4h': y_reg_val[:, 1],
                'return_24h': y_reg_val[:, 2],
                'direction_1h': y_clf_val[:, 0],
                'direction_4h': y_clf_val[:, 1]
            }
        ),
        callbacks=callbacks,
        verbose=1,
        shuffle=False
    )

    print("✅ Entrenamiento completado!")
else:
    print("✅ El entrenamiento ya está completo según los checkpoints existentes")

# Obtener historial completo del entrenamiento
full_history = advanced_checkpoint.training_history if hasattr(advanced_checkpoint, 'training_history') else history.history

# =============================================
# EVALUACIÓN Y ANÁLISIS DEL MODELO
# =============================================

def evaluate_model_comprehensive(model, X_test, y_reg_test, y_clf_test):
    """Evaluación comprehensiva del modelo"""
    # Predecir
    predictions = model.predict(X_test)

    # Evaluación de regresión
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    print("=== EVALUACIÓN DE REGRESIÓN ===")
    horizons = ['1H', '4H', '24H']

    for i, horizon in enumerate(horizons):
        y_true_reg = y_reg_test[:, i]
        y_pred_reg = predictions[i].flatten()

        mse = mean_squared_error(y_true_reg, y_pred_reg)
        mae = mean_absolute_error(y_true_reg, y_pred_reg)
        r2 = r2_score(y_true_reg, y_pred_reg)

        print(f"Horizonte {horizon}:")
        print(f"  MSE: {mse:.6f}")
        print(f"  MAE: {mae:.6f}")
        print(f"  R²: {r2:.4f}")

    # Evaluación de clasificación
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    print("\n=== EVALUACIÓN DE CLASIFICACIÓN ===")
    directions = ['1H', '4H']

    for i, direction in enumerate(directions):
        pred_probs = predictions[i + 3].flatten()
        pred_binary = (pred_probs > 0.5).astype(int)

        y_true_clf = y_clf_test[:, i]

        acc = accuracy_score(y_true_clf, pred_binary)
        precision = precision_score(y_true_clf, pred_binary, zero_division=0)
        recall = recall_score(y_true_clf, pred_binary, zero_division=0)
        f1 = f1_score(y_true_clf, pred_binary, zero_division=0)

        print(f"Dirección {direction}:")
        print(f"  Accuracy: {acc:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1:.4f}")

        cm = confusion_matrix(y_true_clf, pred_binary)
        print(f"  Matriz de confusión:\n{cm}")

print("📊 Evaluando modelo...")
evaluate_model_comprehensive(model, X_test, y_reg_test, y_clf_test)

# =============================================
# BACKTESTING Y SIMULACIÓN DE TRADING
# =============================================

class TradingSimulator:
    def __init__(self, model, df, preprocessor):
        self.model = model
        self.df = df
        self.preprocessor = preprocessor

    def generate_signals(self, X_data, threshold=0.6):
        """Generar señales de trading"""
        predictions = self.model.predict(X_data)
        direction_probs = predictions[3]
        signals = np.where(direction_probs > threshold, 1,
                          np.where(direction_probs < (1 - threshold), -1, 0))

        return signals.flatten(), direction_probs.flatten()

    def simulate_trading(self, initial_capital=10000, transaction_cost=0.0002):
        """Simular estrategia de trading"""
        signals, probabilities = self.generate_signals(X_test)

        test_start_idx = len(self.df) - len(X_test) - SEQUENCE_LENGTH
        prices = self.df['close'].iloc[test_start_idx:test_start_idx + len(signals)].values

        capital = initial_capital
        position = 0
        portfolio_value = []
        trades = []

        for i in range(len(signals)):
            current_price = prices[i]

            if signals[i] == 1 and position <= 0:
                if position < 0:
                    capital += abs(position) * current_price * (1 - transaction_cost)
                    position = 0

                units = capital * 0.98 / current_price
                position = units
                capital -= units * current_price * (1 + transaction_cost)
                trades.append(('BUY', current_price, probabilities[i]))

            elif signals[i] == -1 and position >= 0:
                if position > 0:
                    capital += position * current_price * (1 - transaction_cost)
                    position = 0

                units = capital * 0.98 / current_price
                position = -units
                capital += units * current_price * (1 - transaction_cost)
                trades.append(('SELL', current_price, probabilities[i]))

            portfolio_value.append(capital + position * current_price)

        if position != 0:
            capital += position * prices[-1] * (1 - transaction_cost)
            portfolio_value[-1] = capital

        return portfolio_value, trades, capital

print("💹 Ejecutando simulación de trading...")
simulator = TradingSimulator(model, df_enhanced, preprocessor)
portfolio_values, trades, final_capital = simulator.simulate_trading()

initial_capital = 10000
total_return = (final_capital - initial_capital) / initial_capital * 100
print(f"📈 Retorno total: {total_return:.2f}%")
print(f"🔢 Número de trades: {len(trades)}")
print(f"💰 Capital final: ${final_capital:.2f}")

# =============================================
# VISUALIZACIONES AVANZADAS
# =============================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_comprehensive_visualizations(full_history, portfolio_values, df_enhanced, trades):
    """Crear visualizaciones comprehensivas"""

    # 1. Métricas de entrenamiento
    fig1 = make_subplots(rows=2, cols=2, subplot_titles=['Loss', 'Direction Accuracy', 'Returns MAE', 'Learning Rate'])

    if 'loss' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['loss'], name='Train Loss'), row=1, col=1)
    if 'val_loss' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['val_loss'], name='Val Loss'), row=1, col=1)

    if 'direction_1h_accuracy' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['direction_1h_accuracy'], name='Train Acc 1H'), row=1, col=2)
    if 'val_direction_1h_accuracy' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['val_direction_1h_accuracy'], name='Val Acc 1H'), row=1, col=2)

    if 'return_1h_mae' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['return_1h_mae'], name='Train MAE 1H'), row=2, col=1)
    if 'val_return_1h_mae' in full_history:
        fig1.add_trace(go.Scatter(y=full_history['val_return_1h_mae'], name='Val MAE 1H'), row=2, col=1)

    fig1.update_layout(height=800, title_text="Métricas de Entrenamiento")
    fig1.show()

    # 2. Performance de trading
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(y=portfolio_values, name='Portfolio Value', line=dict(color='blue')))

    buy_trades = [t for t in trades if t[0] == 'BUY']
    sell_trades = [t for t in trades if t[0] == 'SELL']

    if buy_trades:
        buy_indices = [i for i, trade in enumerate(trades) if trade[0] == 'BUY']
        fig2.add_trace(go.Scatter(x=buy_indices, y=[portfolio_values[i] for i in buy_indices],
                             mode='markers', name='Buy', marker=dict(color='green', size=8)))

    if sell_trades:
        sell_indices = [i for i, trade in enumerate(trades) if trade[0] == 'SELL']
        fig2.add_trace(go.Scatter(x=sell_indices, y=[portfolio_values[i] for i in sell_indices],
                             mode='markers', name='Sell', marker=dict(color='red', size=8)))

    fig2.update_layout(title=f"Simulación de Trading - Retorno: {total_return:.2f}%",
                      xaxis_title="Tiempo", yaxis_title="Valor del Portfolio")
    fig2.show()

print("📊 Generando visualizaciones...")
create_comprehensive_visualizations(full_history, portfolio_values, df_enhanced, trades)

# =============================================
# GUARDAR MODELO Y RECURSOS FINALES
# =============================================

# Guardar modelo final
model.save('gbpusd_trading_model_final.h5')

# Guardar preprocessor
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Guardar feature columns
with open('feature_columns.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

# Guardar configuración
config = {
    'EPOCHS': EPOCHS,
    'BATCH_SIZE': BATCH_SIZE,
    'SEQUENCE_LENGTH': SEQUENCE_LENGTH,
    'LEARNING_RATE': LEARNING_RATE,
    'DROPOUT_RATE': DROPOUT_RATE,
    'L2_REGULARIZATION': L2_REGULARIZATION,
    'final_epoch': EPOCHS,
    'total_return': total_return,
    'timestamp': datetime.now().isoformat()
}

with open('training_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("💾 Modelo y recursos guardados exitosamente!")
print("🎯 Entrenamiento completado!")

1 GPUs disponibles
Dataset después del feature engineering: (67539, 69)
Training set: (47235, 60, 62)
Validation set: (6748, 60, 62)
Test set: (13496, 60, 62)
Number of devices: 1
🔄 Checkpoint encontrado: /content/drive/MyDrive/Colab/AI_MODEL/checkpoints/model_epoch_40.h5
📅 Continuando desde época: 40
⚠️ Falló la carga normal del modelo: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}
➡️ Intentando fallback: cargar con compile=False y recompilar manualmente...
✅ Modelo recompilado con compile=False -> compile(...)
✅ Modelo cargado desde checkpoint
🎯 Configuración de entrenamiento:
   - Épocas totales: 200
   - Época inicial: 40
   - Batch size: 2560
   - Checkpoint cada: 10 épocas
   - Learning rate: 0.001
🚀 Iniciando entrenamiento...
Epoch 41/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 624ms/step - direct

19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - direction_1h_accuracy: 0.6040 - direction_1h_binary_accuracy: 0.6040 - direction_1h_loss: 0.6613 - direction_4h_accuracy: 0.7062 - direction_4h_binary_accuracy: 0.7062 - direction_4h_loss: 0.5646 - loss: 2.2503 - return_1h_loss: 2.4793e-04 - return_1h_mae: 0.0104 - return_1h_mse: 2.4831e-04 - return_24h_loss: 2.8249e-04 - return_24h_mae: 0.0116 - return_24h_mse: 2.8289e-04 - return_4h_loss: 3.9098e-04 - return_4h_mae: 0.0125 - return_4h_mse: 3.9152e-04 - val_direction_1h_accuracy: 0.5052 - val_direction_1h_binary_accuracy: 0.5052 - val_direction_1h_loss: 0.7300 - val_direction_4h_accuracy: 0.5230 - val_direction_4h_binary_accuracy: 0.5230 - val_direction_4h_loss: 0.8515 - val_loss: 2.8456 - val_return_1h_loss: 7.5436e-05 - val_return_1h_mae: 0.0078 - val_return_1h_mse: 8.0626e-05 - val_return_24h_loss: 6.7984e-05 - val_return_24h_mae: 0.0063 - val_return_24h_mse: 7.0323e-05 - val_return_4h_loss: 1.5827e-04 - val_return_4h_mae: 0.0093 - val_retur

19/19 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - direction_1h_accuracy: 0.5902 - direction_1h_binary_accuracy: 0.5902 - direction_1h_loss: 0.6694 - direction_4h_accuracy: 0.6906 - direction_4h_binary_accuracy: 0.6906 - direction_4h_loss: 0.5859 - loss: 2.2975 - return_1h_loss: 3.2884e-05 - return_1h_mae: 0.0041 - return_1h_mse: 3.2935e-05 - return_24h_loss: 6.7593e-05 - return_24h_mae: 0.0061 - return_24h_mse: 6.7578e-05 - return_4h_loss: 5.3373e-05 - return_4h_mae: 0.0054 - return_4h_mse: 5.3440e-05 - val_direction_1h_accuracy: 0.4950 - val_direction_1h_binary_accuracy: 0.4950 - val_direction_1h_loss: 0.7205 - val_direction_4h_accuracy: 0.4999 - val_direction_4h_binary_accuracy: 0.4999 - val_direction_4h_loss: 0.7858 - val_loss: 2.7023 - val_return_1h_loss: 5.7951e-06 - val_return_1h_mae: 0.0019 - val_return_1h_mse: 5.9777e-06 - val_return_24h_loss: 5.2917e-05 - val_return_24h_mae: 0.0054 - val_return_24h_mse: 5.7013e-05 - val_return_4h_loss: 3.4113e-05 - val_return_4h_mae: 0.0046 - val_retur

19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 493ms/step - direction_1h_accuracy: 0.6037 - direction_1h_binary_accuracy: 0.6037 - direction_1h_loss: 0.6625 - direction_4h_accuracy: 0.7019 - direction_4h_binary_accuracy: 0.7019 - direction_4h_loss: 0.5699 - loss: 2.2577 - return_1h_loss: 1.7303e-06 - return_1h_mae: 8.6920e-04 - return_1h_mse: 1.7292e-06 - return_24h_loss: 4.7720e-05 - return_24h_mae: 0.0050 - return_24h_mse: 4.7713e-05 - return_4h_loss: 6.2985e-06 - return_4h_mae: 0.0016 - return_4h_mse: 6.2945e-06 - val_direction_1h_accuracy: 0.4969 - val_direction_1h_binary_accuracy: 0.4969 - val_direction_1h_loss: 0.7143 - val_direction_4h_accuracy: 0.5090 - val_direction_4h_binary_accuracy: 0.5090 - val_direction_4h_loss: 0.7934 - val_loss: 2.7001 - val_return_1h_loss: 2.8408e-06 - val_return_1h_mae: 0.0012 - val_return_1h_mse: 3.0396e-06 - val_return_24h_loss: 5.7963e-05 - val_return_24h_mae: 0.0057 - val_return_24h_mse: 6.1806e-05 - val_return_4h_loss: 1.0593e-05 - val_return_4h_mae: 0.0023 - val

✅ Checkpoint guardado en época 50
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 458ms/step - direction_1h_accuracy: 0.6062 - direction_1h_binary_accuracy: 0.6062 - direction_1h_loss: 0.6590 - direction_4h_accuracy: 0.7065 - direction_4h_binary_accuracy: 0.7065 - direction_4h_loss: 0.5677 - loss: 2.2469 - return_1h_loss: 1.6744e-06 - return_1h_mae: 8.4331e-04 - return_1h_mse: 1.6732e-06 - return_24h_loss: 4.6617e-05 - return_24h_mae: 0.0050 - return_24h_mse: 4.6618e-05 - return_4h_loss: 7.5464e-06 - return_4h_mae: 0.0019 - return_4h_mse: 7.5508e-06 - val_direction_1h_accuracy: 0.4988 - val_direction_1h_binary_accuracy: 0.4988 - val_direction_1h_loss: 0.7440 - val_direction_4h_accuracy: 0.5139 - val_direction_4h_binary_accuracy: 0.5139 - val_direction_4h_loss: 0.9322 - val_loss: 2.9942 - val_return_1h_loss: 3.1046e-06 - val_return_1h_mae: 0.0013 - val_return_1h_mse: 3.3534e-06 - val_return_24h_loss: 5.1053e-05 - val_return_24h_mae: 0.0053 - val_return_24h_mse: 5.5230e-05 - val_return_4h_loss: 1.0498e-05

19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 521ms/step - direction_1h_accuracy: 0.6117 - direction_1h_binary_accuracy: 0.6117 - direction_1h_loss: 0.6563 - direction_4h_accuracy: 0.7168 - direction_4h_binary_accuracy: 0.7168 - direction_4h_loss: 0.5488 - loss: 2.2132 - return_1h_loss: 1.7361e-06 - return_1h_mae: 8.6974e-04 - return_1h_mse: 1.7350e-06 - return_24h_loss: 3.9147e-05 - return_24h_mae: 0.0045 - return_24h_mse: 3.9122e-05 - return_4h_loss: 7.3768e-06 - return_4h_mae: 0.0018 - return_4h_mse: 7.3787e-06 - val_direction_1h_accuracy: 0.4935 - val_direction_1h_binary_accuracy: 0.4935 - val_direction_1h_loss: 0.7179 - val_direction_4h_accuracy: 0.5166 - val_direction_4h_binary_accuracy: 0.5166 - val_direction_4h_loss: 0.7931 - val_loss: 2.6967 - val_return_1h_loss: 2.5001e-06 - val_return_1h_mae: 0.0011 - val_return_1h_mse: 2.6766e-06 - val_return_24h_loss: 4.9951e-05 - val_return_24h_mae: 0.0052 - val_return_24h_mse: 5.3783e-05 - val_return_4h_loss: 1.0599e-05 - val_return_4h_mae: 0.0023 - va

✅ Checkpoint guardado en época 60
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 501ms/step - direction_1h_accuracy: 0.6151 - direction_1h_binary_accuracy: 0.6151 - direction_1h_loss: 0.6533 - direction_4h_accuracy: 0.7249 - direction_4h_binary_accuracy: 0.7249 - direction_4h_loss: 0.5378 - loss: 2.1912 - return_1h_loss: 1.7949e-06 - return_1h_mae: 8.8580e-04 - return_1h_mse: 1.7927e-06 - return_24h_loss: 4.4745e-05 - return_24h_mae: 0.0049 - return_24h_mse: 4.4734e-05 - return_4h_loss: 6.8366e-06 - return_4h_mae: 0.0017 - return_4h_mse: 6.8340e-06 - val_direction_1h_accuracy: 0.4964 - val_direction_1h_binary_accuracy: 0.4964 - val_direction_1h_loss: 0.7322 - val_direction_4h_accuracy: 0.5147 - val_direction_4h_binary_accuracy: 0.5147 - val_direction_4h_loss: 0.8829 - val_loss: 2.8811 - val_return_1h_loss: 4.5595e-06 - val_return_1h_mae: 0.0016 - val_return_1h_mse: 4.8391e-06 - val_return_24h_loss: 5.7475e-05 - val_return_24h_mae: 0.0056 - val_return_24h_mse: 6.2168e-05 - val_return_4h_loss: 1.8225e-05

✅ Checkpoint guardado en época 70
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 465ms/step - direction_1h_accuracy: 0.6540 - direction_1h_binary_accuracy: 0.6540 - direction_1h_loss: 0.6207 - direction_4h_accuracy: 0.7866 - direction_4h_binary_accuracy: 0.7866 - direction_4h_loss: 0.4453 - loss: 1.9888 - return_1h_loss: 1.5240e-06 - return_1h_mae: 7.8955e-04 - return_1h_mse: 1.5228e-06 - return_24h_loss: 3.9190e-05 - return_24h_mae: 0.0045 - return_24h_mse: 3.9172e-05 - return_4h_loss: 8.3273e-06 - return_4h_mae: 0.0020 - return_4h_mse: 8.3294e-06 - val_direction_1h_accuracy: 0.4904 - val_direction_1h_binary_accuracy: 0.4904 - val_direction_1h_loss: 0.7258 - val_direction_4h_accuracy: 0.5087 - val_direction_4h_binary_accuracy: 0.5087 - val_direction_4h_loss: 0.8440 - val_loss: 2.7825 - val_return_1h_loss: 2.4523e-06 - val_return_1h_mae: 0.0010 - val_return_1h_mse: 2.6333e-06 - val_return_24h_loss: 5.4998e-05 - val_return_24h_mae: 0.0056 - val_return_24h_mse: 5.8946e-05 - val_return_4h_loss: 1.3022e-05

✅ Checkpoint guardado en época 80
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 470ms/step - direction_1h_accuracy: 0.7071 - direction_1h_binary_accuracy: 0.7071 - direction_1h_loss: 0.5562 - direction_4h_accuracy: 0.8396 - direction_4h_binary_accuracy: 0.8396 - direction_4h_loss: 0.3523 - loss: 1.7211 - return_1h_loss: 2.2160e-06 - return_1h_mae: 0.0010 - return_1h_mse: 2.2165e-06 - return_24h_loss: 3.6768e-05 - return_24h_mae: 0.0043 - return_24h_mse: 3.6741e-05 - return_4h_loss: 5.6109e-06 - return_4h_mae: 0.0015 - return_4h_mse: 5.6100e-06 - val_direction_1h_accuracy: 0.4988 - val_direction_1h_binary_accuracy: 0.4988 - val_direction_1h_loss: 0.7767 - val_direction_4h_accuracy: 0.5087 - val_direction_4h_binary_accuracy: 0.5087 - val_direction_4h_loss: 0.9908 - val_loss: 3.1161 - val_return_1h_loss: 3.1832e-06 - val_return_1h_mae: 0.0013 - val_return_1h_mse: 3.3768e-06 - val_return_24h_loss: 5.3078e-05 - val_return_24h_mae: 0.0054 - val_return_24h_mse: 5.7453e-05 - val_return_4h_loss: 1.1142e-05 - v

💾 Modelo y recursos guardados exitosamente!
🎯 Entrenamiento completado!
